# <u>Face Recognition System

Nowadays there are many ways of authenticating yourself, like using password, retina scan, fingerprint etc. Face can also be used for this purpose. In this notebook we will make a face recognition system using Siamese network.
This is different from face verification where the task is to know whether given two input images are same or not.
Here the task is see whether the given input image is of any person who is registered with the system or not. There can be multiple users registered with the system.

The advantage of **Siamese Network** is that it allows a way to do this sort of verification task with very little user data, as it is quite unreasonable to train using thousands of images for each user. Here we will be using **FaceNet Model**.

FaceNet learns a neural network that encodes a face image into a vector of 128 numbers. So by comparing two such vectors, we can then determine if two pictures are of the same person.

In [1]:
from keras.models import Sequential
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D, AveragePooling2D
from keras.layers.merge import Concatenate
from keras.layers.core import Lambda, Flatten, Dense
from keras.initializers import glorot_uniform
from keras.engine.topology import Layer
from keras import backend as K
from keras.models import load_model
# K.set_image_data_format('channels_first')
import pickle
import cv2
import os.path
import os
import numpy as np
from numpy import genfromtxt
import pandas as pd
from utility import *
from webcam_utility_v2 import *
import tensorflow as tf


config = tf.compat.v1.ConfigProto(gpu_options=tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.5))
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)
tf.compat.v1.keras.backend.set_session(session)

## Model
The model makes an encoding vector consisting of 128 numbers for the input image. Two encodings are compared and if the two encodings are similar then we say that the two images are of the same person otherwise they are different. 
The model uses **Triplet loss function**. The aim is to minimize this function.

In [2]:
# triplet loss function
#  y_pred - list containing three objects:
#         anchor(None, 128) -- encodings for the anchor images
#         positive(None, 128) -- encodings for the positive images
#         negative(None, 128) -- encodings for the negative images
def triplet_loss(y_true, y_pred, alpha = 0.2):
    anchor, positive, negative = y_pred[0], y_pred[1], y_pred[2]
    
    # triplet formula components
    pos_dist = tf.reduce_sum( tf.square(tf.subtract(y_pred[0], y_pred[1])) )
    neg_dist = tf.reduce_sum( tf.square(tf.subtract(y_pred[0], y_pred[2])) )
    basic_loss = pos_dist - neg_dist + alpha
    
    loss = tf.maximum(basic_loss, 0.0)
   
    return loss

### Loading the Model
The model outputs a vector of 128 numbers which represent encoding for the given input image. We will be using this encoding vector for comparing two images.
#### Input
- This network takes as input 96x96 RGB image as its input. Specifically, inputs a tensor of shape $(m, n_C, n_H, n_W)$ , where $n_C$ = channel.

#### Output
- A matrix of shape **(m, 128)** where the 128 numbers are the encoding values for $ith$ image.

In [3]:
# load the model
FRmodel = load_model('models/model.h5', custom_objects={'triplet_loss': triplet_loss})

### User Database

We will create a database of registered. For this we will use a simple dictionary and map each registered user with his/her face encoding.

In [4]:
# initialize the user database
def ini_user_database():
    # check for existing database
    if os.path.exists('database/user_dict.pickle'):
        with open('database/user_dict.pickle', 'rb') as handle:
            user_db = pickle.load(handle)   
    else:
        # make a new one
        # we use a dict for keeping track of mapping of each person with his/her face encoding
        user_db = {}
        # create the directory for saving the db pickle file
        os.makedirs('database')
        with open('database/user_dict.pickle', 'wb') as handle:
            pickle.dump(user_db, handle, protocol=pickle.HIGHEST_PROTOCOL)   
    return user_db

In [5]:
# adds a new user face to the database using his/her image stored on disk using the image path
def add_user_img_path(user_db, FRmodel, name, img_path):
    if name not in user_db: 
        user_db[name] = img_to_encoding(img_path, FRmodel)
        # save the database
        with open('database/user_dict.pickle', 'wb') as handle:
                pickle.dump(user_db, handle, protocol=pickle.HIGHEST_PROTOCOL)
        print('User ' + name + ' added successfully')
    else:
        print('The name is already registered! Try a different name.........')

In [11]:
# adds a new user using image taken from webcam
def add_user_webcam(user_db, FRmodel, name):
    # we can use the webcam to capture the user image then get it recognized
    face_found = detect_face(user_db, FRmodel)

    if face_found:
        resize_img("saved_image/1.jpg")
        if name not in user_db:
            add_user_img_path(user_db, FRmodel, name, "saved_image/1.jpg")
        else:
            print('The name is already registered! Try a different name.........')
    else:
        print('There was no face found in the visible frame. Try again...........')

In [12]:
# deletes a registered user from database
def delete_user(user_db, name):
    popped = user_db.pop(name, None)
    
    if popped is not None:
        print('User ' + name + ' deleted successfully')
        # save the database
        with open('database/user_dict.pickle', 'wb') as handle:
                pickle.dump(user_db, handle, protocol=pickle.HIGHEST_PROTOCOL)
    elif popped == None:
        print('No such user !!')

### Add or delete user 

In [8]:
# we use a dict for keeping track of mapping of each person with his/her face encoding
user_db = ini_user_database()

In [9]:
# add a user
add_user_img_path(user_db, FRmodel, "khoi", "images/My face.jpg")

User khoi added successfully


In [14]:
delete_user(user_db, "khoi")

User khoi deleted successfully


In [15]:
add_user_webcam(user_db, FRmodel, "khoi")

User khoi added successfully


### Putting everything together
For making this face recognition system we are going to take the input image, find its encoding and then see if there is any similar encoding in the database or not. We define a threshold value to decide whether the two images are similar or not based on the similarity of their encodings.

In [19]:
# recognize the input user face encoding by checking for it in the database
def find_face(image_path, database, model, threshold = 0.6):
    # find the face encodings for the input image
    encoding = img_to_encoding(image_path, model)
    
    min_dist = 99999
    # loop over all the recorded encodings in database 
    for name in database:
        # find the similarity between the input encodings and claimed person's encodings using L2 norm
        dist = np.linalg.norm(np.subtract(database[name], encoding) )
        # check if minimum distance or not
        if dist < min_dist:
            min_dist = dist
            identity = name

    if min_dist > threshold:
        print("User not in the database.")
        identity = 'Unknown Person'
    else:
        print ("Hi! " + str(identity) + ", L2 distance: " + str(min_dist))
        
    return min_dist, identity

In [29]:
# detects faces in realtime from webcam feed
def detect_face_realtime(database, model, threshold=0.6):
    detector = MTCNN()
    text = ''
    font = cv2.FONT_HERSHEY_SIMPLEX
    save_loc = r'saved_image/1.jpg'
    capture_obj = cv2.VideoCapture(0)
    capture_obj.set(3, 640)  # WIDTH
    capture_obj.set(4, 480)  # HEIGHT

    face_cascade = cv2.CascadeClassifier(r'haarcascades/haarcascade_frontalface_default.xml')
    print('**************** Enter "q" to quit **********************')
    prev_time = time.time()
    while(True):

        # capture_object frame-by-frame
        ret, frame = capture_obj.read()
        # mirror the frame
        # frame = cv2.flip(frame, 1, 0)

        # Our operations on the frame come here
        # gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        # detect face
        faces = detector.detect_faces(frame)
        # faces = face_cascade.detectMultiScale(gray, 1.3, 5)

        # Display the resulting frame
        if faces:
            for face in faces:
                x, y, w, h = face['box']
                # required region for the face
                roi_color = frame[y-90:y+h+70, x-50:x+w+50]
                if roi_color.any():
                    # save the detected face
                    cv2.imwrite(save_loc, roi_color)
                else:
                    pass

                # keeps track of waiting time for face recognition
                curr_time = time.time()

                if curr_time - prev_time >= 3:
                    img = cv2.imread(save_loc)
                    if img is not None:
                        resize_img(save_loc)

                        min_dist, identity, registered = find_face_realtime(
                            save_loc, database, model, threshold)

                        if min_dist <= threshold and registered:
                            # for putting text overlay on webcam feed
                            text = 'Hello ' + identity
                            print('Hello ' + identity + '!')
                        else:
                            text = 'Unknown user'
                            print('Unknown user' + ' detected !')
                        print('Distance:' + str(min_dist))
                    # save the time when the last face recognition task was done
                    prev_time = time.time()

                # draw a rectangle bounding the face
                cv2.rectangle(frame, (x-10, y-70),
                              (x+w+20, y+h+40), (15, 175, 61), 4)
                cv2.putText(frame, text, (50, 50), font, 1.8, (158, 11, 40), 3)

        # display the frame with bounding rectangle
        cv2.imshow('frame', frame)

        # close the webcam when 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # release the capture_object
    capture_obj.release()
    cv2.destroyAllWindows()

In [23]:
# takes an input image and performs face recognition on it
def do_face_recognition(user_db, FRmodel, threshold=0.7, save_loc="saved_image/1.jpg"):
    # we can use the webcam to capture the user image then get it recognized
    face_found = detect_face(user_db, FRmodel)

    if face_found:
        resize_img("saved_image/1.jpg")
        find_face("saved_image/1.jpg", user_db, FRmodel, threshold)
    else:
        print('There was no face found in the visible frame. Try again...........')

### <u>Run the face recognition system

In [30]:
# Using webcam
detect_face_realtime(user_db, FRmodel, threshold=0.7)

**************** Enter "q" to quit **********************
Hello khoi!
Distance:0.67613655


In [25]:
# using image on disk
do_face_recognition(user_db, FRmodel, threshold = 0.7, save_loc = "images/My face.jpg")

Hi! khoi, L2 distance: 0.45375872
